In [1]:
#%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it
import ipyparallel as ipp

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [4]:
# Create subdirectory /synthetic_data if it does not exist
path = "./synthetic_data"

# Check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

Folder ./synthetic_data already exists


In [5]:
# Setting up the cluster
cluster = ipp.Cluster(n=52)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(52)
rc.ids
dv = rc[:]

Using existing profile dir: '/Users/stigomholt/.ipython/profile_default'
Starting 52 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1739898818-rwpi', profile='default', controller=<running>, engine_sets=['1739898819'])>

  0%|          | 0/52 [00:00<?, ?engine/s]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [6]:
with dv.sync_imports():
    from random import normalvariate
    import numpy as np

importing normalvariate from random on engine(s)
importing numpy on engine(s)


In [7]:
a = 10
b = 5
dv.push({"a": a, "b": b})

<AsyncResult(_push): pending>

In [8]:
# To ensure the variables are on engine 0
rc[0]["a"]

10

In [9]:
rc[50]["b"]

5

### Calculate the initial sizes of all six population categories

In [10]:
%%px
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

In [11]:
%%px
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags, det_type
):
    # Assumed winter survival parameters for all six population categories
    if det_type == 'det_high':
        w_survival = [
        normalvariate(0.8 + 0.04 * 2, 0),
        normalvariate(0.92 + 0.03 * 2, 0),
        normalvariate(0.93 + 0.03 * 2, 0),
        normalvariate(0.8 + 0.04 * 2, 0),
        normalvariate(0.87 + 0.04 * 2, 0),
        normalvariate(0.93 + 0.03 * 2, 0),
    ]
    
    if det_type == 'det_low':
        w_survival = [
        normalvariate(0.8 - 0.04 * 2, 0),
        normalvariate(0.92 - 0.03 * 2, 0),
        normalvariate(0.93 - 0.03 * 2, 0),
        normalvariate(0.8 - 0.04 * 2, 0),
        normalvariate(0.87 - 0.04 * 2, 0),
        normalvariate(0.93 - 0.03 * 2, 0),
    ]
    
    if det_type == 'det_mean':
        w_survival = [
        normalvariate(0.8, 0),
        normalvariate(0.92, 0),
        normalvariate(0.93, 0),
        normalvariate(0.8, 0),
        normalvariate(0.87, 0),
        normalvariate(0.93, 0),
    ]
    
    fraction_female_calves_surviving_winter = w_survival[0]
    fraction_young_hinds_surviving_winter = w_survival[1]
    fraction_adult_hinds_surviving_winter = w_survival[2]
    fraction_male_calves_surviving_winter = w_survival[3]
    fraction_young_stags_surviving_winter = w_survival[4]
    fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [12]:
%%px
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [13]:
%%px
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors, det_type):

    # Assumed fertility parameters for young and adult hinds
    if det_type == 'det_high':
        calves_per_young_hind = normalvariate(0.4 + 2 * 0.05, 0)
        calves_per_adult_hind = normalvariate(0.975 + 2 * 0.008, 0)
    
    if det_type == 'det_low':
        calves_per_young_hind = normalvariate(0.4 - 2 * 0.05, 0)
        calves_per_adult_hind = normalvariate(0.975 - 2 * 0.008, 0)

    if det_type == 'det_mean':
        calves_per_young_hind = normalvariate(0.4, 0)
        calves_per_adult_hind = normalvariate(0.975, 0)
    
    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [14]:
%%px
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1 or adult_stags < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [15]:
%%px
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [16]:
%%px
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [17]:
%%px
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        det_type
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors, det_type)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Transform harvest percentages to numbers

In [18]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

### Scripts for L2_norm_seen_deer analysis

In [19]:
%%px
def L2_norm_seen_deer(
    population_log,
    empirical_observations,
    number_of_years,
):
    """
    Calculating the minimum L2 norm between synthetic data and
    reported data
    """
    import numpy as np
    # Establish the two time series to be compared
    seen_deer_e = np.array(empirical_observations)
    before_hunt_record = [x[17] for x in population_log]
    seen_deer_p = np.array(before_hunt_record)
    
    if seen_deer_p[-1] > 10:
        # Find the scaling factor that minimises L2 norm and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
    else:
        scaling_factor = 100.
    
    seen_deer_p_scaled = scaling_factor * seen_deer_p

    # Find minimum Residual Sum Square value
    L2_norm_min = (np.sum((seen_deer_p_scaled - seen_deer_e) ** 2))**(1/2)
    
     # Pick out last record (2023 one)
    last_year_record = population_log[number_of_years-1][:]
    last_year_record.append(L2_norm_min)

    return last_year_record

### Main ipy parallel script

In [20]:
@dv.remote(block=True)
def ipy_master():
    import numpy as np

    # Initial preparations
    array_top_hits = np.zeros((number_of_top_hits, 32))
    new_val = np.arange(10000, 10000 + number_of_top_hits, 1)
    array_top_hits[:, array_top_hits.shape[1] - 1] = new_val

    # Start going through states
    for state in init_chunk:
        population_log = []

        # Calculate size of each category in the initial population December 31 year -1
        (
            female_calves_survivors,
            young_hinds_survivors,
            adult_hinds_survivors,
            male_calves_survivors,
            young_stags_survivors,
            adult_stags_survivors,
        ) = calculate_initial_size_of_each_category(state)

        # Record initial sizes of the six categories
        female_calves_initially = female_calves_survivors
        young_hinds_initially = young_hinds_survivors
        adult_hinds_initially = adult_hinds_survivors
        male_calves_initially = male_calves_survivors
        young_stags_initially = young_stags_survivors
        adult_stags_initially = adult_stags_survivors

        # Run through all years in the time series for each initial state
        for j in range(number_of_years):
            (
                total_population_in_spring,
                number_of_migrants,
                migration_percentage,
                hinds_per_stag_before_hunting,
                total_population_before_hunting,
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
                total_population_after_hunting,
                hinds_per_stag_after_hunting,
                fraction_female_calves_surviving_winter,
                fraction_young_hinds_surviving_winter,
                fraction_adult_hinds_surviving_winter,
                fraction_male_calves_surviving_winter,
                fraction_young_stags_surviving_winter,
                fraction_adult_stags_surviving_winter,
                calves_per_young_hind,
                calves_per_adult_hind,
            ) = project_population_development(
                number_harvested_female_calves[j],
                number_harvested_young_hinds[j],
                number_harvested_adult_hinds[j],
                number_harvested_male_calves[j],
                number_harvested_young_stags[j],
                number_harvested_adult_stags[j],
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
                state[2],
                j,
            )

            new_record = [
                state[0],
                state[1],
                state[2],
                female_calves_initially,
                young_hinds_initially,
                adult_hinds_initially,
                male_calves_initially,
                young_stags_initially,
                adult_stags_initially,
                first_year + j,
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
                total_population_in_spring,
                total_population_before_hunting,
                total_population_after_hunting,
                hinds_per_stag_before_hunting,
                hinds_per_stag_after_hunting,
                migration_percentage,
                number_of_migrants,
                fraction_female_calves_surviving_winter,
                fraction_young_hinds_surviving_winter,
                fraction_adult_hinds_surviving_winter,
                fraction_male_calves_surviving_winter,
                fraction_young_stags_surviving_winter,
                fraction_adult_stags_surviving_winter,
                calves_per_young_hind,
                calves_per_adult_hind,
            ]

            population_log.append(new_record)

        # Do L2_norm fitting of the number_of_years population trajectory
        if municipality == "Surnadal":
            last_year_record = L2_norm_seen_deer(
                population_log,
                seen_deer_obs_farmland,
                number_of_years,
            )
        else:
            last_year_record = L2_norm_seen_deer(
                population_log,
                seen_deer_obs,
                number_of_years,
            )

        # Find max L2 norm in array_top_hits and exchange if better fit
        if last_year_record[-1] < array_top_hits[np.argmax(array_top_hits[:, -1]), -1]:
            array_top_hits[np.argmax(array_top_hits[:, -1]), :] = last_year_record

    return array_top_hits

In [21]:
%%time

# Figure S5 uses only 'det_mean' data. The 'det_high' and 'det_low' data
# were used in Figure S5 in a previous version of the Supplementary
det_type = "det_mean"  #'det_high' 'det_low' 'det_mean'
# Number of replicates of each initial condition
number_of_top_hits = 2000  # from each engine
dv.push(dict(number_of_top_hits=number_of_top_hits, det_type=det_type))

# Using all six municipalties
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
data_municipality = [
    data_Averoy,
    data_Tingvoll,
    data_Surnadal,
    data_Sunndal,
    data_Vestnes,
    data_Laerdal,
]

for q in range(len(municipalities)):
    [
        municipality,
        first_year,
        last_year,
        number_of_years,
        years,
        seen_deer_obs,
        seen_deer_obs_forest,
        seen_deer_obs_farmland,
        hinds_per_stag_obs,
        total_harvest,
        total_quota,
        fraction_female_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_male_calves_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
        spring_counts,
    ] = data_municipality[q]

    dv.push(
        dict(
            municipality=municipality,
            first_year=first_year,
            last_year=last_year,
            number_of_years=number_of_years,
            years=years,
            seen_deer_obs=seen_deer_obs,
            seen_deer_obs_forest=seen_deer_obs_forest,
            seen_deer_obs_farmland=seen_deer_obs_farmland,
            hinds_per_stag_obs=hinds_per_stag_obs,
            total_harvest=total_harvest,
            fraction_female_calves_harvested=fraction_female_calves_harvested,
            fraction_young_hinds_harvested=fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested=fraction_adult_hinds_harvested,
            fraction_male_calves_harvested=fraction_male_calves_harvested,
            fraction_young_stags_harvested=fraction_young_stags_harvested,
            fraction_adult_stags_harvested=fraction_adult_stags_harvested,
            spring_counts=spring_counts,
        )
    )

    # To ensure the variables are on engines
    # print(rc[5]['years'],rc[45]['total_harvest'])

    # Setting up all initial conditions to be run with calibrated init_pop
    # Make all combinations of the six initial conditions
    initial_population = np.arange(900, 4000, 10)
    # initial_hinds_per_stag = np.array(
    #    [1.7]
    # )  # uses the inverse of the estimated stags_per_hind threshold
    initial_hinds_per_stag = np.arange(1.3, 2.2, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    # stags_per_hind_threshold = np.array(
    #    [0.58]
    # )  # not really initial condition, but initial premise
    stags_per_hind_threshold = np.arange(0.5, 0.8, 0.05)

    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    dv.scatter("init_chunk", all_initial_states)
    # To ensure the variables are on engines
    # check = rc[0]['init_chunk']
    # print("len(init_chunk):",len(check))

    # Transform reported hunting percentages for all years to numbers
    (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    ) = transform_harvested_percentages_to_numbers(
        total_harvest,
        fraction_female_calves_harvested,
        fraction_male_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
    )

    dv.push(
        dict(
            number_harvested_female_calves=number_harvested_female_calves,
            number_harvested_young_hinds=number_harvested_young_hinds,
            number_harvested_adult_hinds=number_harvested_adult_hinds,
            number_harvested_male_calves=number_harvested_male_calves,
            number_harvested_young_stags=number_harvested_young_stags,
            number_harvested_adult_stags=number_harvested_adult_stags,
        )
    )

    # To ensure the variables are on engines
    # print(rc[10]['number_harvested_young_hinds'],rc[45]['number_harvested_adult_stags'])

    # Getting array_top_hits from all engines in a 3d array that needs to be flattened
    array_top_hits = np.array(ipy_master())
    aggregated_arr = array_top_hits.reshape(-1, array_top_hits.shape[-1])

    # Make pandas dataframe for given municipality
    column_values = [
        "init_pop",
        "init_hps",
        "sh_threshold",
        "i_fcalves",
        "i_yhinds",
        "i_ahinds",
        "i_mcalves",
        "i_ystags",
        "i_astags",
        "obs_year",
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "spring_pop",
        "tot_pop_bh",
        "tot_pop_ah",
        "hps_bh",
        "hps_ah",
        "mig%",
        "num_migs",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
        "L2_norm",
    ]
    pop_data = pd.DataFrame(data=aggregated_arr, columns=column_values)
    pop_data = pop_data.astype(
        {
            "init_pop": "int",
            "obs_year": "int",
            "i_fcalves": "int",
            "i_yhinds": "int",
            "i_ahinds": "int",
            "i_mcalves": "int",
            "i_ystags": "int",
            "i_astags": "int",
            "f_calves": "int",
            "y_hinds": "int",
            "a_hinds": "int",
            "m_calves": "int",
            "y_stags": "int",
            "a_stags": "int",
            "spring_pop": "int",
            "tot_pop_bh": "int",
            "tot_pop_ah": "int",
            "num_migs": "int",
        }
    ).round(16)

    # Sort the data frame
    pop_data_sorted = pop_data.sort_values(by=["L2_norm"]).reset_index(drop=True)

    # Storing all top hits to allow variation in the number of top hits later on
    save_dir = "synthetic_data/"
    if det_type == "det_high":
        pop_data_sorted.to_pickle(
            save_dir + "top_hits_" + municipality + "_seen_deer_det_par_H" + ".pkl"
        )

    if det_type == "det_low":
        pop_data_sorted.to_pickle(
            save_dir + "top_hits_" + municipality + "_seen_deer_det_par_L" + ".pkl"
        )

    if det_type == "det_mean":
        pop_data_sorted.to_pickle(
            save_dir + "top_hits_" + municipality + "_seen_deer_det_par_M" + ".pkl"
        )

    print("Municipality = ", municipality)
    pd.set_option("display.max_columns", None)
    display(pop_data_sorted)
    print()
    print()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,L2_norm
0,2500,1.3,0.50,437,82,835,437,296,409,2023,184,91,445,175,118,226,1161,1532,1241,1.639691,1.556987,21.629594,72,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.987141
1,2500,1.3,0.50,437,82,835,437,353,353,2023,184,91,445,175,118,226,1161,1532,1241,1.639691,1.556987,21.629594,72,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.987141
2,2500,1.3,0.50,437,82,835,437,254,452,2023,184,91,445,175,118,226,1161,1532,1241,1.639691,1.556987,21.629594,72,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.987141
3,2500,1.3,0.50,437,82,835,437,211,494,2023,184,91,445,175,118,226,1161,1532,1241,1.639691,1.556987,21.629594,72,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.987141
4,2500,1.3,0.50,437,82,835,437,183,522,2023,184,91,445,175,118,226,1161,1532,1241,1.639691,1.556987,21.629594,72,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.987141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103995,2480,1.4,0.65,421,219,735,421,368,313,2023,194,97,468,185,124,319,1289,1679,1388,1.376486,1.272083,16.699693,71,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.995579
103996,2430,1.3,0.60,352,204,770,352,405,345,2023,171,82,412,162,109,259,1141,1489,1198,1.453740,1.342248,18.862135,68,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.995579
103997,2390,1.5,0.60,394,240,720,394,345,294,2023,176,85,424,167,112,266,1167,1524,1233,1.453900,1.345492,18.672952,69,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.995580
103998,2430,1.5,0.50,413,250,712,413,346,295,2023,191,95,461,182,122,234,1195,1578,1287,1.640007,1.560192,21.448515,74,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.995580




Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,L2_norm
0,3990,2.1,0.5,399,194,1967,399,185,844,2023,500,287,1259,487,312,620,2927,3971,3468,1.635885,1.657244,14.674960,118,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.236218
1,3990,2.1,0.5,399,194,1967,399,205,823,2023,500,287,1259,487,312,620,2927,3971,3468,1.635885,1.657244,14.674960,118,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.236233
2,3990,2.1,0.5,399,194,1967,399,226,803,2023,500,287,1259,487,312,620,2927,3971,3468,1.635885,1.657244,14.674960,118,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.236248
3,3990,2.1,0.5,399,194,1967,399,247,782,2023,500,287,1259,487,312,620,2927,3971,3468,1.635885,1.657244,14.674960,118,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.236263
4,3990,2.1,0.5,399,194,1967,399,267,761,2023,500,287,1259,487,312,620,2927,3971,3468,1.635885,1.657244,14.674960,118,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.236279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103995,3960,2.0,0.5,415,500,1585,415,563,479,2023,335,179,856,322,204,419,2070,2819,2316,1.629816,1.660503,14.567412,83,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.300236
103996,3990,2.1,0.8,478,534,1520,478,528,449,2023,332,177,850,319,202,707,2339,3093,2590,1.174472,1.129452,8.712205,74,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.300241
103997,3940,2.1,0.5,433,541,1540,433,535,456,2023,334,178,855,321,204,418,2068,2816,2313,1.629794,1.660516,14.567030,83,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.300248
103998,3990,2.0,0.7,438,497,1576,438,560,477,2023,338,181,864,325,206,620,2278,3039,2536,1.295267,1.265004,10.121031,77,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.300248




Municipality =  Surnadal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,L2_norm
0,3950,1.6,0.50,691,142,1437,691,493,493,2023,505,304,1273,493,288,604,2841,3917,3469,1.673544,1.766499,8.165175,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.416534
1,3950,1.6,0.50,691,142,1437,691,414,572,2023,505,304,1273,493,288,604,2841,3917,3469,1.673544,1.766499,8.165175,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.416534
2,3950,1.6,0.50,691,142,1437,691,473,513,2023,505,304,1273,493,288,604,2841,3917,3469,1.673544,1.766499,8.165175,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.416534
3,3950,1.6,0.50,691,142,1437,691,217,770,2023,505,304,1273,493,288,604,2841,3917,3469,1.673544,1.766499,8.165175,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.416534
4,3950,1.6,0.50,691,142,1437,691,236,750,2023,505,304,1273,493,288,604,2841,3917,3469,1.673544,1.766499,8.165175,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.416534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103995,3970,1.3,0.60,436,367,1382,436,727,619,2023,497,299,1254,485,283,730,2919,4000,3552,1.478357,1.531908,4.828712,41,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.419265
103996,3970,1.4,0.65,516,411,1302,516,661,563,2023,498,300,1256,486,284,800,2983,4074,3626,1.396758,1.435790,3.530738,32,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.419265
103997,3940,1.6,0.55,610,401,1271,610,564,480,2023,517,312,1302,505,296,689,2965,4073,3625,1.569339,1.637975,6.548178,54,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.419265
103998,3890,1.7,0.50,622,449,1215,622,529,450,2023,505,304,1273,493,288,604,2842,3918,3470,1.673540,1.766477,8.166379,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.419265




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,L2_norm
0,2920,2.0,0.50,292,140,1417,292,420,358,2023,478,290,1255,466,277,617,2771,3835,3387,1.665742,1.726651,8.369382,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.159065
1,2910,2.0,0.50,291,139,1412,291,419,356,2023,470,285,1235,458,272,607,2727,3776,3328,1.665978,1.728008,8.259377,59,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.159074
2,2920,2.0,0.50,292,155,1401,292,420,358,2023,475,288,1247,463,275,613,2753,3811,3363,1.665838,1.727205,8.324457,60,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.159075
3,2920,2.0,0.50,292,140,1417,292,404,373,2023,478,290,1255,466,277,617,2771,3835,3387,1.665742,1.726651,8.369382,61,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.159077
4,2910,2.0,0.50,291,139,1412,291,403,372,2023,470,285,1235,458,272,607,2727,3776,3328,1.665978,1.728008,8.259377,59,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.159087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103995,3050,1.9,0.55,305,431,1166,305,454,387,2023,483,294,1268,471,281,691,2856,3939,3491,1.562785,1.606051,6.677265,53,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.160348
103996,3430,1.3,0.50,343,372,1178,343,644,548,2023,458,277,1206,446,264,593,2667,3695,3247,1.666319,1.729975,8.100467,56,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.160348
103997,3050,2.0,0.50,366,417,1128,366,417,355,2023,458,277,1207,446,264,593,2668,3697,3249,1.666312,1.729931,8.104002,56,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.160349
103998,3060,1.9,0.50,306,352,1251,306,455,388,2023,509,311,1331,497,298,655,2932,4052,3604,1.664925,1.722010,8.747143,67,0.72,0.86,0.87,0.72,0.79,0.87,0.3,0.959,0.160349




CPU times: user 5min 16s, sys: 10min 15s, total: 15min 31s
Wall time: 2h 18min 44s


In [22]:
# stop cluster syncronously
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 99262, 'identifier': 'ipcontroller-1739898818-rwpi-99258'}
Stopping engine(s): 1739898819
engine set stopped 1739898819: {'engines': {'0': {'exit_code': 0, 'pid': 99275, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 99281, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 99282, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 99284, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 99286, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 99289, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 99292, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 99296, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 99299, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 99303, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 99310, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 99316, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 99322, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 99328, 'identifier': '13'}, '14': {'ex

In [23]:
cluster._is_running()

False